In [1]:
import pandas as pd
import sqlite3
import csv
import os
from os.path import isfile, join

In [25]:
def create_database():
    con = sqlite3.connect("DataBase/BinanceExchangeData3.db")
    cur = con.cursor()
    # Create Spot Table
    cur.execute("CREATE TABLE IF NOT EXISTS Spot_1m(open_time, open, high, low, close, volume, close_time, quote_volume, no_of_trade, taker_buy_volume, taker_buy_quote_volume,ignore)")
    con.commit()
    con.close()

In [4]:
data_directory = 'Data/spot/BTCUSDT/1m/'
files_to_inject = os.listdir(data_directory)

In [6]:
## refresh files
files_to_inject = os.listdir(data_directory)

In [7]:
for i in files_to_inject:

    with open(data_directory + i,
              'r') as fin:  # `with` statement available in 2.5+
        # csv.DictReader uses first line in file for column headings by default
        dr = csv.DictReader(fin)  # comma is default delimiter
        to_db = [(i['open_time'], i['open'], i['high'], i['low'], i['close'], i['volume'],
                  i['close_time'], i['quote_volume'], i['count'], i['taker_buy_volume'], i['taker_buy_quote_volume'], i['ignore'])
                 for i in dr]

    cur.executemany(
        "INSERT INTO Spot_1m (open_time, open, high, low, close, volume, close_time, quote_volume, count, taker_buy_volume, taker_buy_quote_volume,ignore) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);",
        to_db)
    con.commit()

In [20]:
res = cur.execute("SELECT * FROM Spot_1m limit 10")

In [21]:
df = pd.read_sql_query("SELECT * FROM Spot_1m", con)

In [22]:
df.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1581206400000,9895.04,9904.32,9893.12,9898.17,19.224579,1581206459999,190288.70846291,437,9.128216,90367.86407137,0
1,1581206460000,9898.35,9901.98,9896.38,9900.24,13.566573,1581206519999,134301.18630029,261,5.222038,51690.37980027,0
2,1581206520000,9900.0,9900.45,9895.36,9897.47,15.790272,1581206579999,156286.961391,269,7.163726,70908.85116667,0
3,1581206580000,9897.47,9898.41,9887.0,9887.01,37.666676,1581206639999,372535.70900667,405,8.985586,88885.85753486,0
4,1581206640000,9888.35,9888.35,9880.75,9882.31,22.191547,1581206699999,219326.59732255,362,9.045279,89398.09425992,0


In [25]:
con.close()